In [1]:
from tqdm import tqdm
from pprint import pprint
import json
import pandas as pd
import sys
sys.path.append('/home/ajassani/iLoveTrace')
from Trace2Tree.trace_to_tree import TraceToTree
from tree_perf import TreePerfAnalyzer

In [2]:
path = '/home/ajassani/trace_data/jan6/MI325/gpt-3-xl_single-gpu_bfloat16_bs5_steps25_det0_prof1_level1/pytorch_profile_rank0.json'
with open(path, 'r') as f:
    data = json.load(f)

events = data['traceEvents']
tree = TraceToTree(events)
tree.build_tree(add_python_func=False)
perf_analyzer = TreePerfAnalyzer(tree)

Building tree with add_python_func=False
Building CPU op tree with add_python_func=False


In [3]:
df_kernel_launchers = perf_analyzer.get_df_kernel_launchers()
df_kernel_launchers.round(2)

,name,Input Dims,total_direct_kernel_time,direct_kernel_count
0,aten::fill_,"((5,), ())",4.89,1
1,aten::mse_loss_backward,"((), (5,), (5,), (), (5,))",4.77,1
2,aten::copy_,"((5,), (5,), ())",5.25,1
3,aten::div,"((5, 2048, 32), ())",5.57,1
4,aten::mm,"((10240, 32), (32, 2048))",26.80,1
...,...,...,...,...
9994,aten::_foreach_addcmul_,"((), (), (), ())",6033.31,100
9995,aten::_foreach_sqrt,"((),)",3677.09,90
9996,aten::_foreach_div_,"((), ())",4721.54,82
9997,aten::_foreach_add_,"((), ())",4687.17,81


In [4]:
# this gives a good perf breakdown at op level
df_kernel_launchers_summary = perf_analyzer.get_df_kernel_launchers_summary(df_kernel_launchers)
df_kernel_launchers_summary.round(2)

,name,total_direct_kernel_time_sum,Count,total_direct_kernel_time_ms,Percentage (%),Cumulative Percentage (%)
13,aten::convolution_backward,363535.30,736,363.54,34.05,34.05
20,aten::miopen_convolution,156527.22,736,156.53,14.66,48.71
21,aten::mm,84654.09,300,84.65,7.93,56.64
19,aten::miopen_batch_norm_backward,59738.87,448,59.74,5.60,62.23
33,flash_attn::_flash_attn_backward,54526.56,25,54.53,5.11,67.34
9,aten::addmm,45762.04,147,45.76,4.29,71.62
18,aten::miopen_batch_norm,43081.60,448,43.08,4.03,75.66
12,aten::clamp_min_,34283.57,416,34.28,3.21,78.87
32,aten::threshold_backward,31333.86,576,31.33,2.93,81.81
14,aten::copy_,30761.00,2858,30.76,2.88,84.69


In [5]:
# We can further get breakdown by shapes for a particular op
df_kernel_launchers_summary_name_shapes = perf_analyzer.get_df_kernel_launchers_summary_by_shape(df_kernel_launchers, "aten::miopen_convolution")
df_kernel_launchers_summary_name_shapes.round(2)

,Input Dims,Total Kernel Time (µs),Count,Mean Kernel Time (µs),Std Kernel Time (µs),Max Direct Kernel Count,Min Direct Kernel Count,Total Kernel Time (ms),Percentage (%),Cumulative Percentage (%)
25,"((5, 896, 59, 91), (896, 896, 1, 1), (), (), (...",34638.80,152,227.89,20.13,4,4,34.64,22.13,22.13
24,"((5, 896, 59, 91), (896, 56, 3, 3), (), (), ()...",19626.73,72,272.59,27.25,5,5,19.63,12.54,34.67
15,"((5, 224, 470, 725), (224, 56, 3, 3), (), (), ...",16607.51,8,2075.94,21.44,5,5,16.61,10.61,45.28
18,"((5, 448, 118, 182), (448, 56, 3, 3), (), (), ...",14882.01,24,620.08,38.24,5,5,14.88,9.51,54.79
19,"((5, 448, 118, 182), (448, 448, 1, 1), (), (),...",8897.00,56,158.87,7.17,1,1,8.90,5.68,60.47
12,"((5, 224, 235, 363), (224, 56, 3, 3), (), (), ...",8788.64,8,1098.58,29.53,5,5,8.79,5.61,66.08
13,"((5, 224, 235, 363), (224, 224, 1, 1), (), (),...",8539.26,24,355.80,25.69,1,1,8.54,5.46,71.54
21,"((5, 448, 235, 363), (448, 56, 3, 3), (), (), ...",7772.03,8,971.50,25.96,5,5,7.77,4.97,76.51
3,"((5, 32, 470, 725), (224, 32, 1, 1), (), (), (...",7202.19,16,450.14,118.97,3,1,7.20,4.60,81.11
14,"((5, 224, 235, 363), (448, 224, 1, 1), (), (),...",6458.96,16,403.69,143.84,3,1,6.46,4.13,85.23
